In [ ]:
import numpy as np
from math import floor
import auxiliary_files.labs_utils as utils

We must implement QAOA as the Custom Quantum Seed.

In [ ]:
def Combine(p1, p2):
    n = len(p1)
    k = np.random.randint(1, n)
    return np.concatenate([p1[:k], p2[k:]])

def Mutate(seq, p_mut):
    seq = seq.copy()
    for i in range(len(seq)):
        if (np.random.random() < p_mut):
            seq[i] *= -1
    return seq

def flip(seq, i):
    s_new = seq.copy()
    s_new[i] *= -1
    return s_new

def energy(seq):
    # Compute E(S) = Σ C_k²
    E = 0
    n = len(seq)
    for k in range(1, n):
        C_k = np.sum(seq[:n-k] * seq[k:])
        E += C_k * C_k
    return E

    
def TabuSearch(seq, max_iter, min_tabu, max_tabu):
    n = len(seq)
    tabu_list = np.zeros(n)
    
    best_seq_ts = seq.copy()
    best_energy_ts = energy(best_seq_ts)
    
    # pivot ← seq
    pivot = seq.copy()
    pivot_energy = energy(pivot)
    
    for t in range(1, max_iter + 1):
        best_i = -1
        best_flip_energy = float('inf')
        best_candidates = []
        
        for i in range(n):
            if tabu_list[i] < t:
                flipped = flip(pivot, i)
                flip_energy = energy(flipped)
                
                if flip_energy < best_flip_energy:
                    best_flip_energy = flip_energy
                    best_candidates = [i]
                elif flip_energy == best_flip_energy:
                    # Randomly choose one if there are multiple
                    best_candidates.append(i)
                    
        if best_candidates:
            best_i = np.random.choice(best_candidates)
        else:
            # If all moves are tabu, pick the least recently tabu'd
            best_i = np.argmin(tabu_list)
            best_flip_energy = energy(flip(pivot, best_i))
        
        # pivot ← flip(pivot, i)
        pivot = flip(pivot, best_i)
        pivot_energy = best_flip_energy
        
        tabu_list[best_i] = t + np.random.randint(min_tabu, max_tabu + 1)
        
        if pivot_energy < best_energy_ts:
            best_seq_ts = pivot.copy()
            best_energy_ts = pivot_energy
    
    return best_seq_ts

    

def MTS(population):

    # stop the while!
    iteration = 0
    max_iterations = 1000
    
    Kp = len(population)     
    Np = len(population[0])
    for i in range(Kp):
        population[i] = TabuSearch(population[i], int(max_iterations // K), 1, max(int(np.sqrt(Np)), 2))

    best_seq = population[0].copy()
    best_energy = energy(best_seq)
    for p in population:
        if energy(p) < best_energy:
            best_seq = p.copy()
            best_energy = energy(p)

    history = [best_energy]
    
    while (best_energy > target_e and iteration < max_iterations):
        iteration += 1
        if (np.random.rand() < p_comb):
            p1_i = np.random.randint(0, Kp)
            p2_i = np.random.randint(0, Kp)
            child = Combine(population[p1_i], population[p2_i])
        else:
            child_i = np.random.randint(0, Kp)
            child = population[child_i].copy()
            
        child = Mutate(child, p_mut)
        child = TabuSearch(child, int(max_iterations // K), 1, max(int(np.sqrt(Np)), 2))
        child_energy = energy(child)
        if(best_energy > child_energy):
            best_seq = child.copy()
            best_energy = child_energy

        history.append(best_energy)
            
        random_idx = np.random.randint(0, K)
        population[random_idx] = child
        
        # Progress
        if iteration % 20 == 0:
            print(f"Iteration {iteration}: Best Energy = {best_energy}")
    
    return best_seq, best_energy, history

We must feed the Quantum Seed to MTS, to see this way is better. 

We also must try MTS with the previous random method.

In [ ]:
K = 2
target_e = 25
p_comb = 0.7
p_mut = 1/N
N = 20
population = [np.random.choice([-1, 1], size = N) for _ in range(K)]
best_seq, best_energy, history = MTS(population)
print(best_seq, best_energy